<a href="https://colab.research.google.com/github/alexlopespereira/minicursoIDP2021/blob/master/ETL_Vacinacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL para carregar as bandeiras dos estados

### 1) Criar o dataframe do pib percapita

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'enap-331414' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq('''
SELECT pop.*, dsc.nome_municipio, pib.pib, pib.pib/pop.populacao as pibpercapita FROM `basedosdados.br_ibge_populacao.municipio` pop
LEFT JOIN `basedosdados.br_ibge_pib.municipio` pib on pop.id_municipio = pib.id_municipio and pib.ano = pop.ano
LEFT JOIN (
    select distinct (sc.id_municipio), sc.nome_municipio from `basedosdados.br_geobr_mapas.setor_censitario_2010` sc
    ) as dsc on dsc.id_municipio = pop.id_municipio
''', project_id=project_id)

df.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita
0,1991,RO,1100015,31981.0,Alta Floresta D'oeste,NaN,NaN
1,1992,RO,1100015,34768.0,Alta Floresta D'oeste,NaN,NaN
2,1993,RO,1100015,37036.0,Alta Floresta D'oeste,NaN,NaN
3,1994,RO,1100015,39325.0,Alta Floresta D'oeste,NaN,NaN
4,1995,RO,1100015,41574.0,Alta Floresta D'oeste,NaN,NaN


### 2) Fazer o download da tabela com URL das bandeiras dos estados

In [3]:
dfb = pd.read_excel("https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/bandeiras/bandeiras.xlsx")
dfb.head()

,ufNome,uf,Bandeira
0,Acre,AC,https://upload.wikimedia.org/wikipedia/commons...
1,Alagoas,AL,https://upload.wikimedia.org/wikipedia/commons...
2,Amapá,AP,https://upload.wikimedia.org/wikipedia/commons...
3,Amazonas,AM,https://upload.wikimedia.org/wikipedia/commons...
4,Bahia,BA,https://upload.wikimedia.org/wikipedia/commons...


In [5]:
dfb['Bandeira'][0]

'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Bandeira_do_Acre.svg/320px-Bandeira_do_Acre.svg.png'

### 3) Fazer o merge com a tabela do pib percapita

In [7]:
dfm = df.merge(dfb, left_on='sigla_uf', right_on='uf')

### 4) Submeter o seu dataframe **substituindo** a tabela já existente do pib percapita no BigQuery. **Não** crie uma tabela nova!! Use o mesmo nome que você usou no seu exercício **3.1**. Por que?

In [8]:
dfm.to_gbq("enapdatasets.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

5it [00:42,  8.58s/it]


### 5) Editar e atualizar o respectivo data source no Data Studio.